In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from pathlib import Path
import os
from data_io import get_book

In [2]:
metadata_filename = 'metadata.csv'
counts_dirname = 'counts'
tokens_dirname = 'tokens'

metadata_df = pd.read_csv(metadata_filename)

filtered_df = metadata_df[(metadata_df.language == "['en']") & (metadata_df.type == 'Text')]

SELECTED_COLUMNS = ['id', 'title', 'author', 'authoryearofbirth', 'authoryearofdeath']
filtered_df = filtered_df.dropna(subset=SELECTED_COLUMNS)
filtered_df = filtered_df[SELECTED_COLUMNS]
filtered_df = filtered_df.reset_index(drop=True)

author_count = filtered_df['author'].value_counts()
many_works_author = author_count[author_count >= 10]
filtered_df = filtered_df[filtered_df.author.isin(many_works_author.index.to_numpy())].reset_index()

'PG8700' in filtered_df.id

False

In [3]:
filtered_df = filtered_df.sample(n=50, random_state=2).reset_index()

docs = []
docs_unavail_pg_ids = []
for pg_id in filtered_df.id:    
    try:
        doc = ' '.join(get_book(pg_id, os.path.join(tokens_dirname), level='tokens'))
        docs.append(doc)
    except:
        docs_unavail_pg_ids.append(pg_id)
        
docs_unavail_pg_ids

['PG6248']

In [4]:
filtered_df = filtered_df[~filtered_df.id.isin(docs_unavail_pg_ids)]
'PG6248' in filtered_df.id

False

In [5]:
filtered_df = filtered_df[['id', 'author']]
filtered_df.insert(2, 'Text', docs, True)
filtered_df

,id,author,Text
0,PG25770,"Seaman, Augusta Huiell",the dragon secret illustration leslie hurried ...
1,PG21292,"Fenn, George Manville",brave and true short stories for children by g...
2,PG41988,"Sienkiewicz, Henryk",let us follow him illustration let us follow h...
3,PG4816,"Motley, John Lothrop",this ebook was produced by david widger note t...
4,PG21301,"Fenn, George Manville",bunyip land a story of adventure in new guinea...
5,PG1103,"Shakespeare, William",this etext file is presented by project gutenb...
6,PG51952,"Moore, Frank Frankfort",provided by the internet archive a journalists...
7,PG51356,"Nietzsche, Friedrich Wilhelm",images generously made available by the hathi ...
8,PG23028,"Laumer, Keith",greylorn by keith laumer keith laumer is a wri...
9,PG9442,"Ward, Humphry, Mrs.",berger and the online distributed proofreading...
